In [1]:
import os
import pickle
import glob
import gzip
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import collections

In [2]:
#2021_mesh
os.chdir('/home/admin_user')
filename_mesh = 'mesh_list_2021.pickle'
with open(filename_mesh, 'rb') as f:
    mesh_list = pickle.load(f)

#メッシュから　メッシュIDと疾患/症状メッシュ、TreeNumber を取り出す
#疾患メッシュ：Cのうち C22（動物)とC23.888（症状）を除外したもの）＋F03（精神障害）
#症状メッシュ：C23.888
mesh_diseases = [descriptor for descriptor in tqdm(mesh_list) for uid in descriptor[-1] if ((uid.startswith("C")==True) & (uid.startswith("C22")!=True)&(uid.startswith("C23.888")!=True)) | (uid.startswith("F03")==True) ]
mesh_symptoms = [descriptor for descriptor in tqdm(mesh_list)  for uid in descriptor[-1] if (uid.startswith("C23.888")==True)]
#さらにメッシュ名だけにする＋重複削除
mesh_diseases= pd.DataFrame(mesh_diseases)[1].drop_duplicates(keep='last').reset_index(drop=True).values.tolist()
mesh_symptoms= pd.DataFrame(mesh_symptoms)[1].drop_duplicates(keep='last').reset_index(drop=True).values.tolist()

len(mesh_list),len(mesh_diseases), len(mesh_symptoms)

<ipython-input-2-e3ed14ff6204>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  mesh_diseases = [descriptor for descriptor in tqdm(mesh_list) for uid in descriptor[-1] if ((uid.startswith("C")==True) & (uid.startswith("C22")!=True)&(uid.startswith("C23.888")!=True)) | (uid.startswith("F03")==True) ]


  0%|          | 0/29917 [00:00<?, ?it/s]

<ipython-input-2-e3ed14ff6204>:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  mesh_symptoms = [descriptor for descriptor in tqdm(mesh_list)  for uid in descriptor[-1] if (uid.startswith("C23.888")==True)]


  0%|          | 0/29917 [00:00<?, ?it/s]

(29917, 4887, 394)

In [3]:
#2011_mesh

filename_mesh = 'mesh_list_2011.pickle'
with open(filename_mesh, 'rb') as f:
    mesh_list_2011 = pickle.load(f)

#メッシュから　メッシュIDと疾患/症状メッシュ、TreeNumber を取り出す
#疾患メッシュ：Cのうち C22（動物)とC23.888（症状）を除外したもの）＋F03（精神障害）
#症状メッシュ：C23.888
mesh_diseases_2011 = [descriptor for descriptor in tqdm(mesh_list_2011) for uid in descriptor[-1] if ((uid.startswith("C")==True) & (uid.startswith("C22")!=True)&(uid.startswith("C23.888")!=True)) | (uid.startswith("F03")==True) ]
mesh_symptoms_2011 = [descriptor for descriptor in tqdm(mesh_list_2011)  for uid in descriptor[-1] if (uid.startswith("C23.888")==True)]
#さらにメッシュ名だけにする＋重複削除
mesh_diseases_2011= pd.DataFrame(mesh_diseases_2011)[1].drop_duplicates(keep='last').reset_index(drop=True).values.tolist()
mesh_symptoms_2011= pd.DataFrame(mesh_symptoms_2011)[1].drop_duplicates(keep='last').reset_index(drop=True).values.tolist()

len(mesh_list_2011),len(mesh_diseases_2011), len(mesh_symptoms_2011)

<ipython-input-3-53d0c44e9195>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  mesh_diseases_2011 = [descriptor for descriptor in tqdm(mesh_list_2011) for uid in descriptor[-1] if ((uid.startswith("C")==True) & (uid.startswith("C22")!=True)&(uid.startswith("C23.888")!=True)) | (uid.startswith("F03")==True) ]


  0%|          | 0/26142 [00:00<?, ?it/s]

<ipython-input-3-53d0c44e9195>:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  mesh_symptoms_2011 = [descriptor for descriptor in tqdm(mesh_list_2011)  for uid in descriptor[-1] if (uid.startswith("C23.888")==True)]


  0%|          | 0/26142 [00:00<?, ?it/s]

(26142, 4475, 328)

In [16]:
zhou_diseases = pd.read_csv("Supplementary Data 1.txt", sep="\t" )["MeSH Disease Term"].values.tolist()
zhou_symptoms = pd.read_csv("Supplementary Data 2.txt", sep="\t" )["MeSH Symptom Term"].values.tolist()
print(len(zhou_diseases), len(zhou_symptoms))
print(len(mesh_diseases_2011), len(mesh_symptoms_2011))

4442 322
4475 328


In [25]:
4475-4442

33

In [47]:
#これまで使っていた2011_疾患メッシュ: Cのうち C22（動物)とC23.888（症状）を除外したもの）＋F03（精神障害）

#これまで使っていた2011_疾患メッシュ(4,475)のうち、 zhou_diseases(4,442)に含まれていないタームとツリー番号
nn_diseaes_1 = []
for md in mesh_diseases_2011:
    if md not in zhou_diseases:
        for mesh in mesh_list_2011:
            if md == mesh[1]:
                nn_diseaes_1.append( [mesh[1], mesh[-1]])                
print(len(nn_diseaes_1))
nn_diseaes_1

62


[['Aneuploidy', ['C23.550.210.050', 'G05.365.590.175.050', 'G05.700.131']],
 ['Bacterial Infections and Mycoses', ['C01']],
 ['Mental Disorders', ['F03']],
 ['Cardiovascular Diseases', ['C14']],
 ['Chromosome Deletion',
  ['C23.550.210.050.500.500',
   'G05.355.600.800.180',
   'G05.365.590.029.530.175',
   'G05.365.590.175.050.500.500',
   'G05.365.590.762.180',
   'G05.700.131.500.500']],
 ['Digestive System Diseases', ['C06']],
 ['Endocrine System Diseases', ['C19']],
 ['Eye Diseases', ['C11']],
 ['Female Urogenital Diseases and Pregnancy Complications', ['C13']],
 ['Fetishism (Psychiatric)', ['F03.800.800.600.350']],
 ['Hemic and Lymphatic Diseases', ['C15']],
 ['Immune System Diseases', ['C20']],
 ['Disorders of Environmental Origin', ['C21']],
 ['Stomatognathic Diseases', ['C07']],
 ['Musculoskeletal Diseases', ['C05']],
 ['Congenital, Hereditary, and Neonatal Diseases and Abnormalities', ['C16']],
 ['Neoplasms', ['C04']],
 ['Neoplasms by Histologic Type', ['C04.557']],
 ['Neopla

In [42]:
#zhou_diseases(4,442)のうち、 選定済みMmesh_diseases_2011(4,475)に含まれていないタームとツリー番号

nn_diseaes_2 = []
for z_d in zhou_diseases:
    if z_d not in mesh_diseases_2011:
        for mesh in mesh_list_2011:
            if z_d == mesh[1]:
                nn_diseaes_2.append( [mesh[1], mesh[-1]])                
print(len(nn_diseaes_2))
nn_diseaes_2

#C22(animal)

29


[['Dog Diseases', ['C22.268']],
 ['Cattle Diseases', ['C22.196']],
 ['Horse Diseases', ['C22.488']],
 ['Cat Diseases', ['C22.180']],
 ['Poultry Diseases', ['C22.131.728']],
 ['Swine Diseases', ['C22.905']],
 ['Sheep Diseases', ['C22.836']],
 ['Fish Diseases', ['C22.362']],
 ['Rodent Diseases', ['C22.795']],
 ['Bird Diseases', ['C22.131']],
 ['Mastitis, Bovine', ['C22.196.581']],
 ['Monkey Diseases', ['C22.735.500']],
 ['Muscular Dystrophy, Animal', ['C22.595']],
 ['Goat Diseases', ['C22.405']],
 ['Lameness, Animal', ['C22.510']],
 ['PC12 Cells', ['A11.251.210.190.750', 'A11.251.860.180.750', 'A11.299.500']],
 ['Hip Dysplasia, Canine', ['C22.268.485']],
 ['Ape Diseases', ['C22.735.050']],
 ['Foot Rot', ['C22.394']],
 ['Parturient Paresis', ['C22.695']],
 ['Disease Models, Animal', ['C22.232', 'E05.598.500', 'E05.599.395.080']],
 ['Primate Diseases', ['C22.735']],
 ['Edema Disease of Swine', ['C22.905.260']],
 ['Porcine Postweaning Multisystemic Wasting Syndrome', ['C22.905.690']],
 ['Wh

In [50]:
#症状も

nn_symptoms_1 = []
for i_1 in mesh_symptoms_2011:
    if i_1 not in zhou_symptoms:
        for term in mesh_list_2011:
            if i_1 == term[1]:
                print(term[1], term[-1])
                nn_symptoms_1.append(i)
len(nn_symptoms_1)

Body Temperature Changes ['C23.888.119']
Signs and Symptoms ['C23.888']
Signs and Symptoms, Digestive ['C23.888.821']
Signs and Symptoms, Respiratory ['C23.888.852']
Neuromuscular Manifestations ['C10.597.613', 'C23.888.592.608']
Urological Manifestations ['C23.888.942']


6

In [51]:
nn_symptoms_2 = []
for i_2 in zhou_symptoms:
    if i_2 not in mesh_symptoms_2011:
        for term in mesh_list_2011:
            if i_2 == term[1]:
                print(term[1], term[-1])
                nn_symptoms_2.append(i)
len(nn_symptoms_2)

0